# Prophet Forecast Notebook

This notebook demonstrates how to:
1. Query historical metrics from Victoria Metrics
2. Use darts Prophet model for forecasting
3. Generate forecasts and save to database

**Configuration:** 
- Connection settings (VM URL, DB credentials) are provided via parameters or environment variables
- Query selector and history days are configured in the query section (section 3)
- Model parameters are configured in the model training section (section 5)


In [ ]:
# Parameters cell (tagged for papermill injection)
# These will be injected by papermill when executed by the job
# When running locally, use environment variables instead
# Note: selector, history_days, forecast parameters, and model parameters
# are hardcoded in the notebook cells where they are used (NOT passed as parameters)
# Database configuration is loaded from YAML config file using VM_JOBS_ENVIRONMENT and VM_JOBS_DB_PASSWORD
vm_query_url = ''
vm_token = ''
vm_jobs_environment = ''
dry_run = True


## 1. Configuration

**Connection settings:** Set via papermill parameters or environment variables.
**Forecasting parameters:** Will be defined in the model training cell (see section 5).


In [ ]:
# Configuration
# Connection settings: from papermill parameters or environment variables
# Database configuration is loaded automatically from YAML config using VM_JOBS_ENVIRONMENT and VM_JOBS_DB_PASSWORD
import os

# Victoria Metrics connection - from parameters/env vars (prefer parameters)
# Parameters are now individual variables (not a dict) due to papermill requirements
VM_QUERY_URL = vm_query_url if vm_query_url else os.getenv('VM_QUERY_URL', 'http://victoria-metrics:8428')
VM_TOKEN = vm_token if vm_token else os.getenv('VM_TOKEN', '')

# Environment - from parameters/env vars (prefer parameters)
VM_JOBS_ENVIRONMENT = vm_jobs_environment if vm_jobs_environment else os.getenv('VM_JOBS_ENVIRONMENT', '')
# Config file path - from environment variable
VM_JOBS_CONFIG_PATH = os.getenv('VM_JOBS_CONFIG_PATH', '')

# Dry run mode: train and plot, but don't save to database
# Handle both bool and string types from papermill
if isinstance(dry_run, bool):
    DRY_RUN = dry_run
elif isinstance(dry_run, str):
    DRY_RUN = dry_run.lower() in ('true', '1', 'yes')
else:
    DRY_RUN = os.getenv('DRY_RUN', 'false').lower() in ('true', '1', 'yes')

print(f"VM Query URL: {VM_QUERY_URL}")
print(f"VM_JOBS_ENVIRONMENT: {VM_JOBS_ENVIRONMENT or 'NOT SET (will use env var)'}")
print(f"Dry Run Mode: {DRY_RUN}")


## 2. Imports and Setup


In [ ]:
import sys
import os
from pathlib import Path

# Add current directory to Python path
current_dir = str(Path.cwd())
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone, date
import warnings
warnings.filterwarnings('ignore')

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Darts imports
from darts import TimeSeries
from darts.models import Prophet as DartsProphet

# Helper modules
from prometheus_api_client import PrometheusConnect
from database_helpers import (
    create_database_connection,
    create_forecast_run_record,
    save_forecasts_to_database
)

print("Imports successful")


In [ ]:
# Set plot style for dry run mode (if enabled)
if DRY_RUN:
    sns.set_style('whitegrid')
    plt.rcParams['figure.figsize'] = (15, 8)
    print("Plotting enabled for dry run mode")


## 3. Connect to Victoria Metrics and Query Data

**Configure your selector and history days in the cell below.**


In [ ]:
# PromQL selector - HARDCODE THIS
SELECTOR = '{job="extractor"}'  # EDIT THIS: Your PromQL selector

# History parameter - HARDCODE THIS
HISTORY_DAYS = 365  # Days of history to fetch

# Connect to Victoria Metrics and query historical data
# Connect to Victoria Metrics and query historical data
headers = {"Authorization": f"Bearer {VM_TOKEN}"} if VM_TOKEN else {}
prom = PrometheusConnect(url=VM_QUERY_URL, headers=headers, disable_ssl=True)
print(f"Connected to Victoria Metrics at {VM_QUERY_URL}")


print(f"\nQuerying: {SELECTOR}")
end_date = datetime.now(timezone.utc)
start_date = end_date - timedelta(days=HISTORY_DAYS)
query_result = prom.custom_query_range(
    query=SELECTOR.replace("'", '"'),  # Ensure double quotes for PromQL
    start_time=start_date,
    end_time=end_date,
    step="24h"
)

print(f"Query range: {start_date.date()} to {end_date.date()}")
print(f"Query returned {len(query_result)} series")


## 4. Parse and Prepare Data

This parses ALL time series returned by the selector query.


In [ ]:
# Parse all series from query result
all_series = []
for item in query_result:
    metric = item.get('metric', {})
    metric_name = metric.get('__name__')
    if not metric_name:
        continue
    labels = {k: v for k, v in metric.items() if k != '__name__'}
    values = item.get('values', [])
    samples = [(datetime.fromtimestamp(float(ts), tz=timezone.utc), float(value)) for ts, value in values]
    if samples:
        all_series.append((samples, {'metric_name': metric_name, 'labels': labels}))

if not all_series:
    raise ValueError("No data found for selector")

print(f"Found {len(all_series)} time series for selector: {SELECTOR}")
print("\nSeries preview:")
for idx, (samples, series_info) in enumerate(all_series[:5]):
    print(f"  {idx+1}. {series_info['metric_name']} {series_info['labels']}")
if len(all_series) > 5:
    print(f"  ... and {len(all_series) - 5} more")


## 5. Train Model and Generate Forecast

**Note:** This processes ALL time series returned by the selector, training a separate model for each series.


In [ ]:
# Model parameters - HARDCODE THESE
FORECAST_HORIZON_DAYS = 20  # Business days to forecast ahead
MIN_HISTORY_POINTS = 30  # Minimum data points required
PROPHET_PARAMS = {
    'yearly_seasonality': False,
    'weekly_seasonality': False,
    'daily_seasonality': False,
    'seasonality_mode': 'additive',
    'changepoint_prior_scale': 0.5,
}
PROPHET_FIT_PARAMS = {}  # Optional fit parameters

# Floor parameter: set to 0 to prevent negative forecasts
# Alternatively, set to a small positive value (e.g., 0.01) if your metrics should never be zero
FORECAST_FLOOR = 0  # Minimum value for forecasts (set to None to disable)

print(f"Forecast horizon: {FORECAST_HORIZON_DAYS} business days")
print(f"Prophet params: {PROPHET_PARAMS}\n")

# Process each series: train model and generate forecast
forecasts_by_series = []

for series_idx, (samples, series_info) in enumerate(all_series):
    print(f"\n{'='*60}")
    print(f"Processing {series_idx + 1}/{len(all_series)}: {series_info['metric_name']}")
    
    try:
        # Prepare business day data (query result already has one value per 24h)
        df_training = pd.DataFrame(samples, columns=['ds', 'y'])
        df_training['ds'] = pd.to_datetime(df_training['ds'], utc=True).dt.tz_localize(None)
        
        if df_training.empty:
            print(f"  ⚠️  Skipping: no data")
            continue
        
        
        if len(df_training) < MIN_HISTORY_POINTS:
            print(f"  ⚠️  Skipping: insufficient data ({len(df_training)} < {MIN_HISTORY_POINTS})")
            continue
        
        print(f"  Training data: {len(df_training)} points, {df_training['ds'].min().date()} to {df_training['ds'].max().date()}")
        
        # Add floor/cap columns if floor is specified (required for Prophet to enforce floor)
        # Prophet requires logistic growth mode and floor/cap columns in the data
        prophet_params = PROPHET_PARAMS.copy()
        if FORECAST_FLOOR is not None:
            # Set growth to logistic (required for floor/cap to work)
            prophet_params['growth'] = 'logistic'
            # Add floor column to training data
            df_training['floor'] = FORECAST_FLOOR
            # Add cap (set to a value above max to avoid constraining upper bound)
            df_training['cap'] = df_training['y'].max() * 1.5 if df_training['y'].max() > FORECAST_FLOOR else FORECAST_FLOOR + 1
        
        # Convert to darts TimeSeries (including floor/cap columns if present)
        series = TimeSeries.from_dataframe(df_training.set_index('ds'))
        
        model = DartsProphet(**prophet_params)
        model.fit(series)
        
        # Generate forecast with uncertainty intervals (use num_samples for probabilistic forecast)
        # num_samples=100 generates 100 sample paths to estimate uncertainty
        # If floor is set via logistic growth, Prophet will automatically respect it
        forecast = model.predict(n=FORECAST_HORIZON_DAYS, num_samples=100)
        forecast_df = pd.DataFrame({
            'ds': forecast.time_index,
            'yhat': forecast.values().flatten()
        })
        
        # Extract 95% prediction intervals (2.5% and 97.5% quantiles)
        if hasattr(forecast, 'quantile_timeseries'):
            try:
                lower = forecast.quantile_timeseries(0.025)  # 2.5% quantile
                upper = forecast.quantile_timeseries(0.975)  # 97.5% quantile
                forecast_df['yhat_lower'] = lower.values().flatten()
                forecast_df['yhat_upper'] = upper.values().flatten()
            except Exception:
                # If quantiles are not available, skip intervals
                pass
        
        print(f"  ✓ Forecast: {len(forecast_df)} predictions")
        
        # Store forecast
        if DRY_RUN:
            forecasts_by_series.append((series_info, forecast_df, df_training))
        else:
            forecasts_by_series.append((series_info, forecast_df))
        
    except Exception as exc:
        print(f"  ✗ Failed: {exc}")
        continue

print(f"\n{'='*60}")
print(f"Successfully forecasted {len(forecasts_by_series)}/{len(all_series)} series")


## 6. Visualize Results (Dry Run Mode)

**Note:** In dry run mode, plots are generated for each series showing historical data and forecasts.


In [ ]:
# Plot forecasts for each series (dry run mode)
if DRY_RUN:
    print(f"\n{'='*60}")
    print("DRY RUN MODE: Generating plots (no database operations)")
    print(f"{'='*60}\n")
    
    for plot_idx, forecast_item in enumerate(forecasts_by_series):
        # Unpack: (series_info, forecast_df, df_training) in dry run mode
        series_info, forecast_df, df_training = forecast_item
        
        plt.figure(figsize=(18, 8))
        
        # Plot historical data
        plt.plot(df_training['ds'], df_training['y'], 
                'ko-', label='Historical Data', linewidth=2, markersize=3, alpha=0.6)
        
        # Plot forecast trend
        plt.plot(forecast_df['ds'], forecast_df['yhat'], 
                'b--', label='Forecast (trend)', linewidth=2.5)
        
        # Plot uncertainty intervals (if available)
        if 'yhat_lower' in forecast_df.columns and 'yhat_upper' in forecast_df.columns:
            plt.fill_between(forecast_df['ds'], 
                           forecast_df['yhat_lower'], 
                           forecast_df['yhat_upper'],
                           alpha=0.2, color='blue', label='Uncertainty Interval')
        
        # Vertical line showing where forecast starts
        last_history_date = df_training['ds'].max()
        plt.axvline(x=last_history_date, color='red', linestyle=':', 
                   linewidth=2, label='Forecast Start', alpha=0.7)
        
        # Title and labels
        title = f"Prophet Forecast: {series_info['metric_name']}"
        if series_info['labels']:
            title += f" {series_info['labels']}"
        plt.title(title, fontsize=16, fontweight='bold')
        plt.xlabel('Date', fontsize=12)
        plt.ylabel('Value', fontsize=12)
        plt.legend(loc='best', fontsize=10)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        print(f"  ✓ Plotted forecast for {series_info['metric_name']}")
    
    print(f"\n{'='*60}")
    print(f"Dry run complete: {len(forecasts_by_series)} series forecasted and plotted")
    print("No data was saved to database.")
else:
    # Create database connection using helper function
    # This automatically loads database config from YAML using VM_JOBS_ENVIRONMENT and VM_JOBS_DB_PASSWORD
    engine, conn = create_database_connection(environment=VM_JOBS_ENVIRONMENT if VM_JOBS_ENVIRONMENT else None, config_path=VM_JOBS_CONFIG_PATH if VM_JOBS_CONFIG_PATH else None)
    
    print("Database connection established")


## 7. Save Forecasts to Database

**Note:** This step is skipped in dry run mode.


In [ ]:
# Save forecasts to database for each series (only if not dry run)
if not DRY_RUN:
    # Create run record once (shared across all series for this selector)
    # Only include intervals if they exist in the forecast
    forecast_types = [{"name": "trend", "field": "yhat"}]
    # Check if any forecast has intervals to determine if we should include them
    has_intervals = any(
        'yhat_lower' in item[1].columns and 'yhat_upper' in item[1].columns
        for item in forecasts_by_series
    )
    if has_intervals:
        forecast_types.extend([
            {"name": "lower", "field": "yhat_lower"},
            {"name": "upper", "field": "yhat_upper"},
        ])
    
    run_id = create_forecast_run_record(
        conn=conn,
        job_id="metrics_forecast_notebooks",
        selection_value=SELECTOR,
        model_type="prophet",
        model_config=PROPHET_PARAMS,
        model_fit_config=PROPHET_FIT_PARAMS,
        history_days=HISTORY_DAYS,
        forecast_horizon_days=FORECAST_HORIZON_DAYS,
        min_history_points=MIN_HISTORY_POINTS,
        config_source="notebook",
    )
    
    print(f"Created forecast run record: run_id={run_id}")
    print(f"\nSaving forecasts for {len(forecasts_by_series)} series...")
    
    total_rows_inserted = 0
    for forecast_item in forecasts_by_series:
        # Unpack: (series_info, forecast_df) in normal mode
        series_info, forecast_df = forecast_item[0], forecast_item[1]
        try:
            rows_inserted, _, _ = save_forecasts_to_database(
                conn=conn,
                metric_name=series_info['metric_name'],
                labels=series_info['labels'],
                forecast_df=forecast_df,
                forecast_types=forecast_types,
                run_id=run_id,  # Link all forecasts to the same parameter record
            )
            total_rows_inserted += rows_inserted
            print(f"  ✓ {series_info['metric_name']}: {rows_inserted} rows saved")
        except Exception as exc:
            print(f"  ✗ {series_info['metric_name']}: Failed to save - {exc}")
    
    print(f"\nTotal: {total_rows_inserted} forecast rows saved to database")
else:
    print("Dry run mode: Skipping database save operations")


In [ ]:
# Close database connection (only if not dry run)
if not DRY_RUN:
    conn.close()
    engine.dispose()
    print("Database connection closed")
